In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from torchvision import transforms, datasets
import torchvision.transforms.functional as fn
from PIL import Image

c:\Users\Leonard\anaconda3\envs\pytorch_nn_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class VegetableClassifier(nn.Module):
    def __init__(self, out_size=15) -> None:
        super(VegetableClassifier, self).__init__()
        self.linear_out_size = out_size
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.network = nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size= (7,7), stride=(2,2), padding=(3,3), bias=False), #maybe add bias?
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64),
        )
        self.avg_maxpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.linear_classifier = nn.Linear(in_features=64, out_features=self.linear_out_size, bias=True)
        
    def forward(self, input):
        input = torch.unsqueeze(input, 0)
        input = self.network(input)
        input = self.avg_maxpool(input)
        input = torch.flatten(input, 1)
        return self.linear_classifier(input)
    
    def fit(self, num_epochs, loss_fn, optimizer, dataloader):
        for epoch in range(num_epochs):
            best_loss = 200000
            for xb, yb in dataloader:
                xb = xb.to(self.device)
                yb = yb.to(self.device)

                optimizer.zero_grad()
                pred = self.forward(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                optimizer.step()

                best_loss = loss if loss < best_loss else best_loss
            print(f"Epoch {epoch}: best_loss: {best_loss}")

                

In [3]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], 
                                                            [0.5, 0.5, 0.5])])

In [4]:
dataset = datasets.ImageFolder('data/Vegetable Images/train', transform=train_transform) # temp changed to test
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
model = VegetableClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.99))
loss_fn = nn.CrossEntropyLoss()

In [10]:
model.fit(num_epochs=20, loss_fn=loss_fn, optimizer=optimizer, dataloader=dataloader)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 32, 3, 224, 224]